In [1]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems
sE, sI, rE, rI, Y  = -50..50, -50..50,  -50..50, -50..50, -50..50
box = sE × sI × rE × rI × Y
using GLMakie, LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function get_valus_vectors(fp)
    Jfp = rate_jac(fp, p, 0.0)
    Jfp_value_vectors = eigen(Jfp)
    Jfp_value = Jfp_value_vectors.values
    Jfp_vectors = Jfp_value_vectors.vectors
    return Jfp_value, Jfp_vectors
end

get_valus_vectors (generic function with 1 method)

In [14]:
τsE = 3.0; γE = 4.0; s0E = 0.15;
τsI = 10.0; γI = 8.0; s0I = 0.1 # 0.3267
# IE = 0.9; 
τrE = 2.0; kE = 5.0; IE = 0.9; wEE = 3.5; wIE = 5.0; θE = 0.2;
τrI = 6.0; kI = 5.0; II = 0.0; wEI = 5.0; wII = 3.0; θI = 0.4;

τY = 0.01;  βY = 0.0;
ythr = 0.5; sEthr = 0.5; kY = 0.01; γY = 0.0

0.0

In [15]:
time = 500.0; tt = 0.0; tstep = 0.001; tspan = (0.0, time)
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [16]:
p = [τsE, γE, s0E, τsI, γI, s0I, τrE, kE, IE, wEE, wIE, θE, τrI, kI, II, wEI, wII, θI, τY, βY, γY, ythr, sEthr, kY];
u0 = [0.0, 0.0, 0.0, 0.0, 0.0]

ds = CoupledODEs(model_braint_rhythms, u0, p, diffeq = integ_set);

In [17]:
fp, _, _ = fixedpoints(ds, box, rate_jac);

In [19]:
fp[1]

5-element SVector{5, Float64} with indices SOneTo(5):
  0.2522329213720189
  0.45021724371685473
  0.034179400342014936
  0.07962627958826166
 -2.7617715536363738e-67

In [8]:
valuesfp1, vectorsfp1 = get_valus_vectors(fp[3])

([-100.0, -1.7779371261620636, -0.8028053921851471, 0.0025466515889394988, 0.41680599642289984], [-0.0008185327177697605 -0.8393629248388502 … -0.2852904284154034 0.5876324122887837; -6.7620377925110675e-9 -0.038351106937266115 … -0.13963072665121126 0.06890990399913219; … ; 6.490785940195281e-6 0.401095807831147 … -0.942769063882444 0.7412059536601635; 0.9999996649690234 0.0 … 0.0 0.0])

In [9]:
valuesfp1

5-element SVector{5, Float64} with indices SOneTo(5):
 -100.0
   -1.7779371261620636
   -0.8028053921851471
    0.0025466515889394988
    0.41680599642289984

In [10]:
Jac = rate_jac(fp[3], p, 0)
eivecs = eigvecs(Jac)
shift = real(eivecs[:, 4])*1e-6 + fp[3]

5-element SVector{5, Float64} with indices SOneTo(5):
 0.6452839487186042
 0.870682381180306
 0.07329021069342598
 0.7449510598956918
 0.0

In [11]:
prob = ODEProblem(model_braint_rhythms, shift, tspan, p);

In [12]:
sol = solve(prob, alg = Vern9(), adaptive = false, dt = 0.001)

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 500002-element Vector{Float64}:
   0.0
   0.001
   0.002
   0.003
   0.004
   0.005
   0.006
   0.007
   0.008
   0.009000000000000001
   ⋮
 499.9929999950886
 499.9939999950886
 499.99499999508856
 499.99599999508854
 499.9969999950885
 499.9979999950885
 499.99899999508847
 499.99999999508844
 500.0
u: 500002-element Vector{SVector{5, Float64}}:
 [0.6452839487186042, 0.870682381180306, 0.07329021069342598, 0.7449510598956918, 0.0]
 [0.6452839487178774, 0.8706823811799503, 0.07329021069316756, 0.7449510598932907, 0.0]
 [0.6452839487171507, 0.8706823811795946, 0.07329021069290914, 0.7449510598908896, 0.0]
 [0.6452839487164239, 0.8706823811792389, 0.07329021069265072, 0.7449510598884885, 0.0]
 [0.6452839487156972, 0.8706823811788832, 0.0732902106923923, 0.7449510598860875, 0.0]
 [0.6452839487149704, 0.8706823811785275, 0.07329021069213389, 0.7449510598836864, 0.0]
 [0.6452839487142437, 0.8706823811781718, 0.0732

In [13]:
tstart, tend = 1, 500000
GLMakie.activate!()
f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 1, 1, 1)

lines!(ax, sol[3, tstart:tend], sol[4, tstart:tend], sol[1, tstart:tend], linewidth = 0.5, color = :deeppink)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)